# Multimatch example

Obligatory imports

In [56]:

import numpy as np
import lsst.daf.base as dafBase
import lsst.daf.persistence as dafPersist



import lsst.afw.display.ds9 as ds9

import matplotlib.pyplot as plt

import lsst.afw.image as afwImage
import lsst.afw.table as afwTable
import lsst.afw.geom as afwGeom

import lsst.afw.display.ds9 as ds9

%matplotlib inline

Instancing the butler in place. DATADIR is the path to the data.

In [ ]:
DATADIR="/renoir_data_02/jpreyes/lsst_data/CFHTLS_master/"
butler = dafPersist.Butler(DATADIR+"/output")

This block gets the source catalog and the calexp exposures to be matched

In [57]:
src1 = butler.get("src", {'visit': 802231 , 'filter':'r' , 'ccd':14})
calexp1 = butler.get("calexp", {'visit': 802231 , 'filter':'r' , 'ccd':14})

src2 = butler.get("src", {'visit': 802230 , 'filter':'r' , 'ccd':14})
calexp2 = butler.get("calexp", {'visit': 802230 , 'filter':'r' , 'ccd':14})

srcs = [(802231,src1), (802230,src2)]
calexps = [calexp1, calexp2]

Multimatch object is created

In [58]:
multi_matches = afwTable.MultiMatch(src1.schema, {'visit':int}, radius=afwGeom.Angle(1./3600., afwGeom.degrees))

Adding catalogs to the multimatch

In [59]:
for visit, catalog in srcs:
    multi_matches.add(catalog, {'visit':visit})
results = multi_matches.finish(removeAmbiguous=False)

Visualization of each images, adding a number on each match

In [60]:
for i, calexp in enumerate(calexps):
    ds9.mtv(calexp, frame=i)
    for result in results:
        if result.get("visit") == srcs[i][0]:
            
            wcs = calexp.getWcs()
            point = wcs.skyToPixel(result.get("coord_ra"), result.get("coord_dec"))
            ds9.dot(str(result.get("object")), point.getX(), point.getY(), size=1, frame=i, ctype=ds9.RED)

